# Проведение экспериментов по настройке модели
## Загрузка модулей

In [1]:
import numpy as np
import pandas as pd
import os
import sys
import pickle

## Загрузка данных

In [2]:
with open("../data/clean_data.pkl", "rb") as f:
    data = pickle.load(f)

data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1935 entries, 1 to 1999
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   battery_power  1935 non-null   int64   
 1   blue           1935 non-null   category
 2   clock_speed    1935 non-null   float64 
 3   dual_sim       1935 non-null   category
 4   fc             1935 non-null   int64   
 5   four_g         1935 non-null   category
 6   int_memory     1935 non-null   int64   
 7   m_dep          1935 non-null   float64 
 8   mobile_wt      1935 non-null   int64   
 9   n_cores        1935 non-null   int64   
 10  pc             1935 non-null   int64   
 11  px_height      1935 non-null   int64   
 12  px_width       1935 non-null   int64   
 13  ram            1935 non-null   int64   
 14  sc_h           1935 non-null   int64   
 15  sc_w           1935 non-null   int64   
 16  talk_time      1935 non-null   int64   
 17  three_g        1935 non-null   categor

In [3]:
numerical_features = data.select_dtypes(exclude="category").columns.drop('price_range')
nominal_features = data.select_dtypes('category').columns
target = 'price_range'

In [4]:
numerical_features, nominal_features

(Index(['battery_power', 'clock_speed', 'fc', 'int_memory', 'm_dep',
        'mobile_wt', 'n_cores', 'pc', 'px_height', 'px_width', 'ram', 'sc_h',
        'sc_w', 'talk_time'],
       dtype='object'),
 Index(['blue', 'dual_sim', 'four_g', 'three_g', 'touch_screen', 'wifi'], dtype='object'))

## Pipeline
### Подготовка

In [5]:
from sklearn.preprocessing import StandardScaler, TargetEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from IPython.display import display


In [6]:
X = data.drop(target, axis=1)
y = data[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.75)

In [7]:
preprocessor = ColumnTransformer(
    transformers=[
        ('numeric', StandardScaler(), numerical_features),
        ('nominal', TargetEncoder(), nominal_features)
])
pipeline = Pipeline([
    ('transform', preprocessor),
    ('classification', RandomForestClassifier())
])

### baseline

In [8]:
from sklearn.metrics import recall_score, precision_score, f1_score, roc_auc_score

In [37]:
def calc_metrics(y_test, predictions, average='weighted'):
    metrics = {}
    metrics["recall"] = recall_score(y_test, predictions, average=average)   
    metrics["precision"] = precision_score(y_test, predictions, average=average)
    metrics["f1"] = f1_score(y_test, predictions, average=average)
    return metrics

def printf_metrics(metrics):
    print(*[f"{m:<9} {v:.5f}" for m, v in metrics.items()], sep='\n')

In [27]:
pipeline.fit(X_train, y_train)
predictions = pipeline.predict(X_test)
print(*[f"{m:<9} {v:.5f}" for m, v in calc_metrics(y_test, predictions).items()], sep='\n')
pipeline

recall    0.85537
precision 0.85730
f1        0.85103


,steps,"[('transform', ...), ('classification', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('numeric', ...), ('nominal', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


### mlflow

In [28]:
import mlflow

In [29]:
TARGET_HOST = "localhost"
TARGET_PORT = 5000
TRACKING_URI = f"http://{TARGET_HOST}:{TARGET_PORT}"
REGISTRY_URI = TRACKING_URI
EXPERIMENT_NAME = 'Mobile Price Classification'
SIGNATURE = mlflow.models.infer_signature(model_input=X_train.head(5))
mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_registry_uri(REGISTRY_URI)

/home/podlubny/Projects/mohov/.venv/lib/python3.11/site-packages/mlflow/types/utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


In [66]:
def log_mlflow_model(
    run_name, estimator, metrics=None, params=None, signature=SIGNATURE, artifacts=None
):
    if (exp := mlflow.get_experiment_by_name(EXPERIMENT_NAME)) is not None:
        experiment_id = exp.experiment_id
    else:
        experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)

    with mlflow.start_run(run_name=run_name, experiment_id=experiment_id) as run:
        run_id = run.info.run_id 
        if artifacts is not None:
            for a in artifacts:
                mlflow.log_artifact(a)
        mlflow.sklearn.log_model(
            estimator, artifact_path="models", signature=signature,
        )
        mlflow.log_metrics(metrics or {})
        mlflow.log_params(params or {})
    
    run = mlflow.get_run(run_id)
    return run.info.status =='FINISHED'

In [31]:
ok = log_mlflow_model(
    'baseline', pipeline, metrics=calc_metrics(y_test, predictions))
assert ok

2025/11/11 21:17:24 INFO mlflow.tracking._tracking_service.client: 🏃 View run baseline at: http://localhost:5000/#/experiments/1/runs/0a8da86789db4f5bb053c2e1d957a67f.
2025/11/11 21:17:24 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/1.


## Генерация новых признаков

In [34]:
from sklearn.preprocessing import PolynomialFeatures, KBinsDiscretizer

In [35]:
def update_data(data, colname_template, transformer, init_features):
    new_data = transformer.fit_transform(data[init_features])
    new_features = [colname_template.format(i) for i in range(new_data.shape[1])]
    data[new_features] = new_data
    return data

In [40]:
poly_features_init = ['sc_h', 'sc_w']
kbins_features_init = ['battery_power', 'n_cores']

poly = Pipeline([
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('scale', StandardScaler())
])

kbins = Pipeline([
    ('kbins', KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')),
    ('scale', StandardScaler())
])

extended_transformer = ColumnTransformer([
    ('numeric', StandardScaler(), numerical_features),
    ('poly', poly, poly_features_init),
    ('kbins', kbins, kbins_features_init),
    ('nominal', TargetEncoder(), nominal_features)
])

rf_extended = RandomForestClassifier()
pipeline_fe = Pipeline([
    ('transform', extended_transformer),
    ('clf', rf_extended)
])

pipeline_fe.fit(X_train, y_train)
predictions = pipeline_fe.predict(X_test)
metrics = calc_metrics(y_test, predictions)
printf_metrics(metrics)
pipeline_fe

/home/podlubny/Projects/mohov/.venv/lib/python3.11/site-packages/sklearn/preprocessing/_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be changed to quantile_method='averaged_inverted_cdf' in scikit-learn version 1.9 to naturally support sample weight equivalence properties by default. Pass quantile_method='averaged_inverted_cdf' explicitly to silence this warning.
  warnings.warn(


recall    0.85950
precision 0.85958
f1        0.85653


,steps,"[('transform', ...), ('clf', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('numeric', ...), ('poly', ...), ...]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [ ]:
new_features_path = '../mlflow/new_features.txt'
X_train_fe_cols = np.array(pipeline_fe.named_steps['transform'].get_feature_names_out())
with open(new_features_path, 'w') as f:
    print(*X_train_fe_cols, sep=',', file=f)

log_mlflow_model(
    'FeatureEngeneering', pipeline_fe,
    metrics=metrics,
    signature=mlflow.models.infer_signature(model_input=X_train.head(5)),
    artifacts=[new_features_path]
)

/home/podlubny/Projects/mohov/.venv/lib/python3.11/site-packages/mlflow/types/utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025/11/11 21:40:56 INFO mlflow.tracking._tracking_service.client: 🏃 View run FeatureEngeneering at: http://localhost:5000/#/experiments/1/runs/6e7df84d36b14b16a22a2c7af9828411.
2025/11/11 21:40:56 INFO mlflow.tracking._tra

True

## Выбор наиболее значимых признаков

In [42]:
from sklearn.feature_selection import SequentialFeatureSelector

In [ ]:
selector = SequentialFeatureSelector(
    RandomForestClassifier(), n_features_to_select=10, direction='forward'
)
sample_X = X_train.sample(frac=0.3, random_state=42)
sample_y = y_train.loc[sample_X.index]
selector.fit(sample_X, sample_y)

selected_features = {i: sample_X.columns[i] for i in selector.get_support(indices=True)}

selected_features_path = '../mlflow/selected_features.txt'
with open(selected_features_path, 'w') as f:
    print(*selected_features.keys(), sep=',', file=f)
    print(*selected_features.values(), sep=',', file=f)

print('selected features:', selected_features)

selected features: {0: 'battery_power', 3: 'dual_sim', 4: 'fc', 5: 'four_g', 11: 'px_height', 12: 'px_width', 13: 'ram', 14: 'sc_h', 17: 'three_g', 18: 'touch_screen'}


In [48]:
class IndexSelector:
    def __init__(self, indices: dict):
        self.indices = list(indices)
        self.features = list(indices.values())
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[:, self.indices]
    
    def fit_transform(self, X, y=None):
        return self.transform(X)

In [49]:
pipeline_fs = Pipeline(steps=[
    ("transformer", extended_transformer), 
    ('feature_selection', IndexSelector(selected_features)),
    ("model", RandomForestClassifier(random_state=42))
])

pipeline_fs.fit(X_train, y_train)
predictions = pipeline_fs.predict(X_test)
metrics = calc_metrics(y_test, predictions)
printf_metrics(metrics)
pipeline_fs

/home/podlubny/Projects/mohov/.venv/lib/python3.11/site-packages/sklearn/preprocessing/_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be changed to quantile_method='averaged_inverted_cdf' in scikit-learn version 1.9 to naturally support sample weight equivalence properties by default. Pass quantile_method='averaged_inverted_cdf' explicitly to silence this warning.
  warnings.warn(


recall    0.29545
precision 0.29207
f1        0.29208


,steps,"[('transformer', ...), ('feature_selection', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('numeric', ...), ('poly', ...), ...]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [50]:
log_mlflow_model(
    'FeatureSelection', pipeline_fs,
    metrics=metrics,
    signature=mlflow.models.infer_signature(X_train, y_train),
    artifacts=[selected_features_path]
)

/home/podlubny/Projects/mohov/.venv/lib/python3.11/site-packages/mlflow/types/utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025/11/11 21:53:35 INFO mlflow.tracking._tracking_service.client: 🏃 View run FeatureSelection at: http://localhost:5000/#/experiments/1/runs/164a4deb2af8434fb731437864a2567a.
2025/11/11 21:53:35 INFO mlflow.tracking._track

True

## Подброр гиперпараметров

In [51]:
import optuna
from sklearn.model_selection import cross_val_score

/home/podlubny/Projects/mohov/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [58]:
import warnings
warnings.filterwarnings('ignore')

In [60]:
def objective(trial: optuna.Trial):
    n_estimators = trial.suggest_int("n_estimators", 50, 200)
    max_depth = trial.suggest_int("max_depth", 3, 25)
    max_features = trial.suggest_float("max_features", 0.1, 1.0)

    pipeline = Pipeline(steps=[
        ("transformer", extended_transformer),
        ("clf", RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            max_features=max_features
        ))
    ])
    score = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='f1_weighted').mean()
    return score

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10, n_jobs=-1)

print("Best parameters:", study.best_params)


[I 2025-11-11 22:20:47,815] A new study created in memory with name: no-name-d9300f57-14ce-400f-a254-786922aa926e
[I 2025-11-11 22:20:52,529] Trial 2 finished with value: 0.8646460291556796 and parameters: {'n_estimators': 50, 'max_depth': 9, 'max_features': 0.2942318941169423}. Best is trial 2 with value: 0.8646460291556796.
[I 2025-11-11 22:20:53,811] Trial 4 finished with value: 0.8874649534584597 and parameters: {'n_estimators': 54, 'max_depth': 16, 'max_features': 0.7345370273752724}. Best is trial 4 with value: 0.8874649534584597.
[I 2025-11-11 22:20:54,108] Trial 7 finished with value: 0.7864450068089642 and parameters: {'n_estimators': 99, 'max_depth': 3, 'max_features': 0.49204917434518725}. Best is trial 4 with value: 0.8874649534584597.
[I 2025-11-11 22:20:54,309] Trial 0 finished with value: 0.8823215386924108 and parameters: {'n_estimators': 60, 'max_depth': 24, 'max_features': 0.792080344422551}. Best is trial 4 with value: 0.8874649534584597.
[I 2025-11-11 22:20:54,665] 

Best parameters: {'n_estimators': 74, 'max_depth': 22, 'max_features': 0.797474612969723}


In [62]:
pipeline_best_params = Pipeline(steps=[
    ("transformer", extended_transformer),
    ("clf", RandomForestClassifier(**study.best_params))
])

pipeline_best_params.fit(X_train, y_train)

predictions = pipeline_best_params.predict(X_test)
metrics = calc_metrics(y_test, predictions)
printf_metrics(metrics)
pipeline_best_params

recall    0.90083
precision 0.90155
f1        0.90003


,steps,"[('transformer', ...), ('clf', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('numeric', ...), ('poly', ...), ...]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [65]:
log_mlflow_model(
    'FeatureEngeneeringBestParams', pipeline_best_params,
    metrics=metrics,
    params=study.best_params,
    signature=mlflow.models.infer_signature(X_train, y_train),
    artifacts=[selected_features_path]
)

2025/11/11 22:27:35 INFO mlflow.tracking._tracking_service.client: 🏃 View run FeatureEngeneeringBestParams at: http://localhost:5000/#/experiments/1/runs/4d4beea10f1349d2b37e66eaf065dfa7.
2025/11/11 22:27:35 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/1.


True

In [68]:
best_final_model = pipeline_best_params

# Объединяем train и test для обучения на всей выборке
X_full = pd.concat([X_train, X_test], axis=0)
y_full = pd.concat([y_train, y_test], axis=0)

best_final_model.fit(X_full, y_full)
signature = mlflow.models.infer_signature(X_full, best_final_model.predict(X_full.head(5)))

used_columns = list(X_full.columns)
with open('production_features.pkl', 'wb') as f:
    pickle.dump(used_columns, f)

params ={
    "model_type": "RandomForestClassifier",
    "training_data": "full_dataset",
    "feature_engineering": "PolynomialFeatures + KBinsDiscretizer + Scaling",
    "total_samples": len(X_full),
    "features_count": len(used_columns)
} | study.best_params
log_mlflow_model(
    'FeatureEngeneeringBestParamsFull', params,
    signature=signature,
    artifacts=['production_features.pkl']
)

2025/11/11 22:38:37 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2025/11/11 22:38:38 INFO mlflow.tracking._tracking_service.client: 🏃 View run FeatureEngeneeringBestParamsFull at: http://localhost:5000/#/experiments/1/runs/f957e2266ba54f04a7bf8ea4b4f32d27.
2025/11/11 22:38:38 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/1.


True